<a href="https://colab.research.google.com/github/jazacmq/Lampornis/blob/main/Lampornis_Ejemplo1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

from google.colab import drive
drive.mount('/content/drive')

# Cargar los CSV
df_indices = pd.read_csv('/content/drive/MyDrive/GEE_Exports/series_temporales_5.csv')
df_temp = pd.read_csv('/content/drive/MyDrive/GEE_Exports/Temperatura_Ejemplo.csv')
df_precip = pd.read_csv('/content/drive/MyDrive/GEE_Exports/Precipitacion_ejemplo.csv')




Mounted at /content/drive


In [ ]:
# Verifica si hay columnas con el mismo nombre en tus datasets
print("Columnas en df_indices:", df_indices.columns.tolist())
print("Columnas en df_temp:", df_temp.columns.tolist())
print("Columnas en df_precip:", df_precip.columns.tolist())

# Busca duplicados específicamente
print("¿Duplicados en df_indices?", df_indices.columns.duplicated().any())
print("¿Duplicados en df_temp?", df_temp.columns.duplicated().any())
print("¿Duplicados en df_precip?", df_precip.columns.duplicated().any())

Columnas en df_indices: ['date', 'NDVI_mean', 'SAVI_mean', 'NDWI_mean', 'cloud_cover', 'system:index']
Columnas en df_temp: ['system:index', 'date', 'system:time_start', 'temperatura_promedio', '.geo']
Columnas en df_precip: ['system:index', 'date', 'precipitacion_diaria', 'system:time_start', '.geo']
¿Duplicados en df_indices? False
¿Duplicados en df_temp? False
¿Duplicados en df_precip? False


In [ ]:
from pickle import TRUE
# Verifica duplicados en la columna 'date' de cada dataset
print("Duplicados en df_indices date:", df_indices['date'].duplicated().any())
print("Duplicados en df_temp date:", df_temp['date'].duplicated().any())
print("Duplicados en df_precip date:", df_precip['date'].duplicated().any())

# Muestra los duplicados específicos
if df_indices['date'].duplicated().any():
    print(df_indices[df_indices['date'].duplicated(keep=False)])
# Encontrar los valores duplicados específicos
duplicados = df_indices[df_indices.duplicated(subset=['date'], keep=False)]
print("Filas duplicadas encontradas:")
print(duplicados.sort_values('date'))

# Ver las frecuencias exactas
conteo_fechas = df_indices['date'].value_counts()
print("\nFrecuencia de cada fecha:")
print(conteo_fechas[conteo_fechas > 1])  # Solo las que se repiten

print(df_indices['date'].value_counts)
df_indices

Duplicados en df_indices date: True
Duplicados en df_temp date: False
Duplicados en df_precip date: False
           date  NDVI_mean  SAVI_mean  NDWI_mean  cloud_cover  \
0    2015-01-18   0.305483   0.458216   0.133960        28.86   
1    2015-04-08   0.227593   0.341384   0.116747        42.51   
2    2015-04-24   0.328473   0.492700   0.155863        34.99   
3    2015-05-26   0.326085   0.489120   0.165276        52.68   
4    2015-06-11   0.260467   0.390695   0.159064        33.63   
..          ...        ...        ...        ...          ...   
169  2019-09-10   0.227718   0.341571   0.121157        53.64   
170  2019-09-26   0.295064   0.442587   0.129371        13.05   
171  2019-10-28   0.304890   0.457326   0.125691        16.04   
173  2019-11-29   0.167989   0.251980   0.089956        19.08   
174  2019-12-15   0.284958   0.427428   0.101092         0.51   

             system:index  
0    LC08_026045_20150118  
1    LC08_026045_20150408  
2    LC08_026045_20150424  
3

,date,NDVI_mean,SAVI_mean,NDWI_mean,cloud_cover,system:index
0,2015-01-18,0.305483,0.458216,0.133960,28.86,LC08_026045_20150118
1,2015-04-08,0.227593,0.341384,0.116747,42.51,LC08_026045_20150408
2,2015-04-24,0.328473,0.492700,0.155863,34.99,LC08_026045_20150424
3,2015-05-26,0.326085,0.489120,0.165276,52.68,LC08_026045_20150526
4,2015-06-11,0.260467,0.390695,0.159064,33.63,LC08_026045_20150611
...,...,...,...,...,...,...
170,2019-09-26,0.295064,0.442587,0.129371,13.05,LC08_026046_20190926
171,2019-10-28,0.304890,0.457326,0.125691,16.04,LC08_026046_20191028
172,2019-11-13,0.229092,0.343631,0.094705,44.62,LC08_026046_20191113
173,2019-11-29,0.167989,0.251980,0.089956,19.08,LC08_026046_20191129


In [ ]:
# Eliminar duplicados manteniendo la primera aparición
df_indices = df_indices.drop_duplicates(subset=['date'], keep='first')

In [ ]:
df_completo = df_indices.merge(df_temp, on='date', how='outer').merge(df_precip, on='date', how='outer')

# 3. Verificar que todo esté bien
print(f"Dataset completo: {df_completo.shape[0]} filas, {df_completo.shape[1]} columnas")
print(f"Fechas únicas en dataset final: {df_completo['date'].nunique()}")

Dataset completo: 1825 filas, 14 columnas
Fechas únicas en dataset final: 1825


In [ ]:
df_completo = df_indices.merge(df_temp, on='date', how='inner').merge(df_precip, on='date', how='inner')

In [ ]:
# Verifica si las fechas son exactamente iguales
print("Ejemplo de fechas en cada dataset:")
print("Índices:", df_indices['date'].iloc[0])
print("Temperatura:", df_temp['date'].iloc[0])
print("Precipitación:", df_precip['date'].iloc[0])

# Puede que tengan diferente formato o zona horaria

Ejemplo de fechas en cada dataset:
Índices: 2015-01-18
Temperatura: 2015-01-01
Precipitación: 2015-01-01


In [ ]:
# Verificar coincidencia exacta de fechas
fechas_indices = set(pd.to_datetime(df_indices['date']))
fechas_temp = set(pd.to_datetime(df_temp['date']))
fechas_precip = set(pd.to_datetime(df_precip['date']))

print("Fechas en índices pero no en temperatura:", len(fechas_indices - fechas_temp))
print("Fechas en temperatura pero no en índices:", len(fechas_temp - fechas_indices))
print("Fechas comunes:", len(fechas_indices & fechas_temp))

Fechas en índices pero no en temperatura: 1
Fechas en temperatura pero no en índices: 1719
Fechas comunes: 97


In [ ]:
# 1. Analizar patrón de valores faltantes
print("Porcentaje de NaN por columna:")
print((df_completo.isnull().sum() / len(df_completo) * 100).round(2))

Porcentaje de NaN por columna:
date                      0.0
NDVI_mean                 0.0
SAVI_mean                 0.0
NDWI_mean                 0.0
cloud_cover               0.0
system:index_x            0.0
system:index_y            0.0
system:time_start_x       0.0
temperatura_promedio    100.0
.geo_x                    0.0
system:index              0.0
precipitacion_diaria      0.0
system:time_start_y       0.0
.geo_y                    0.0
dtype: float64


In [ ]:
# 1. Preparar DataFrame de temperatura
df_temp['date'] = pd.to_datetime(df_temp['date'])
df_temp = df_temp.set_index('date').sort_index()

# 2. Interpolar temperatura
temp_interp_series = df_temp['temperatura_promedio'].interpolate(method='time')

# 3. Preparar DataFrame principal
df_completo = df_indices.merge(df_precip, on='date', how='inner')
df_completo['date'] = pd.to_datetime(df_completo['date'])

# 4. Asignar temperaturas interpoladas
df_completo['temperatura_promedio'] = df_completo['date'].map(temp_interp_series)

# 5. Eliminar NaN residuales
df_completo = df_completo.dropna(subset=['temperatura_promedio'])

In [ ]:
df_completo

,date,NDVI_mean,SAVI_mean,NDWI_mean,cloud_cover,system:index_x,system:index_y,precipitacion_diaria,system:time_start,.geo,temperatura_promedio
0,2015-01-18,0.305483,0.458216,0.133960,28.86,LC08_026045_20150118,20150118,0.311412,1.421539e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",15.338452
1,2015-04-08,0.227593,0.341384,0.116747,42.51,LC08_026045_20150408,20150408,0.000000,1.428451e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",25.818415
2,2015-04-24,0.328473,0.492700,0.155863,34.99,LC08_026045_20150424,20150424,4.345995,1.429834e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",29.149337
3,2015-05-26,0.326085,0.489120,0.165276,52.68,LC08_026045_20150526,20150526,0.000000,1.432598e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",26.710845
4,2015-06-11,0.260467,0.390695,0.159064,33.63,LC08_026045_20150611,20150611,0.000000,1.433981e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",25.916901
...,...,...,...,...,...,...,...,...,...,...,...
93,2018-05-02,0.322464,0.483687,0.142574,46.46,LC08_026046_20180502,20180502,3.784436,1.525219e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",28.105824
94,2018-11-10,0.042411,0.063616,0.155880,66.88,LC08_026046_20181110,20181110,0.000000,1.541808e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",22.917324
95,2019-03-18,0.030428,0.045641,0.124831,38.12,LC08_026046_20190318,20190318,2.335728,1.552867e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",28.753024
96,2019-06-22,0.326045,0.489059,0.139528,42.45,LC08_026046_20190622,20190622,5.231847,1.561162e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",28.085249


In [ ]:
# Verificación rápida de calidad de datos
print("=== VERIFICACIÓN FINAL DEL DATASET ===")
print(f"📊 Número de filas: {len(df_completo)}")
print(f"✅ Porcentaje de valores no nulos: {(1 - df_completo.isnull().mean().max()) * 100:.1f}%")
print(f"📅 Rango temporal: {df_completo['date'].min()} a {df_completo['date'].max()}")
print(f"🌡️ Variables disponibles: {list(df_completo.columns)}")

# Verificar distribución básica
print("\n📈 Estadísticas descriptivas:")
print(df_completo[['NDVI_mean', 'SAVI_mean', 'NDWI_mean', 'temperatura_promedio', 'precipitacion_diaria']].describe())

=== VERIFICACIÓN FINAL DEL DATASET ===
📊 Número de filas: 97
✅ Porcentaje de valores no nulos: 100.0%
📅 Rango temporal: 2015-01-02 00:00:00 a 2019-12-15 00:00:00
🌡️ Variables disponibles: ['date', 'NDVI_mean', 'SAVI_mean', 'NDWI_mean', 'cloud_cover', 'system:index_x', 'system:index_y', 'precipitacion_diaria', 'system:time_start', '.geo', 'temperatura_promedio']

📈 Estadísticas descriptivas:
       NDVI_mean  SAVI_mean  NDWI_mean  temperatura_promedio  \
count  97.000000  97.000000  97.000000             97.000000   
mean    0.270096   0.405136   0.144012             24.539170   
std     0.098271   0.147403   0.032084              4.301846   
min    -0.001194  -0.001791   0.067101             11.805248   
25%     0.227593   0.341384   0.116775             22.946566   
50%     0.305483   0.458216   0.148713             25.136125   
75%     0.330566   0.495840   0.167734             26.710845   
max     0.413895   0.620831   0.233118             37.205543   

       precipitacion_diaria  

In [ ]:
# Esto se hace SOBRE TU DATAFRAME COMPLETO, antes de separar X e y
df_completo['mes'] = df_completo['date'].dt.month
df_completo['trimestre'] = df_completo['date'].dt.quarter
df_completo['dia_del_año'] = df_completo['date'].dt.dayofyear

# Crear tendencias (diferencias con valor anterior)
df_completo['NDVI_tendencia'] = df_completo['NDVI_mean'].diff()
df_completo['temp_tendencia'] = df_completo['temperatura_promedio'].diff()

# Promedios móviles (tendencia suavizada)
df_completo['NDVI_media_movil'] = df_completo['NDVI_mean'].rolling(window=3, min_periods=1).mean()

In [ ]:
umbral_floracion = df_completo['NDVI_mean'].quantile(0.70)  # Percentil 85%

In [ ]:
df_completo['floracion'] = (df_completo['NDVI_mean'] > umbral_floracion).astype(int)

# Condiciones para floración (ajusta estos valores según tu especie/región)
condiciones_floracion = (
    (df_completo['temperatura_promedio'] >18) &  # Alta vegetación
    (df_completo['precipitacion_diaria'] > 10)    # Alta vegetación ajustada# Temporada de primavera
)

df_completo.loc[condiciones_floracion, 'floracion'] = 1

print("Distribución de floración:")
print(df_completo['floracion'].value_counts())
print(f"Porcentaje de floración: {df_completo['floracion'].mean()*100:.1f}%")

Distribución de floración:
floracion
0    64
1    32
Name: count, dtype: int64
Porcentaje de floración: 33.3%


In [ ]:
df_completo['precipitacion_diaria'].min()

0.0

In [ ]:
# Definir variables predictoras (INCLUYENDO las nuevas)
variables_predictoras = [
    'NDVI_mean', 'SAVI_mean', 'NDWI_mean',
    'temperatura_promedio', 'precipitacion_diaria',
    'cloud_cover', 'mes', 'trimestre', 'dia_del_año',
    'NDVI_tendencia', 'temp_tendencia', 'NDVI_media_movil'
]

# Eliminar filas con NaN en las nuevas variables (si las hay)
df_completo = df_completo.dropna(subset=variables_predictoras)
df_completo = df_completo.reset_index(drop=True)

# Crear X e y
X = df_completo[variables_predictoras]
y = df_completo['floracion']  # Asegúrate de haber creado esta variable

In [ ]:
# Ordenar por fecha para validación temporal
df_completo = df_completo.sort_values('date')
X = X.loc[df_completo.index]  # Mantener el mismo orden
y = y.loc[df_completo.index]

# Dividir de manera temporal (no aleatoria)
split_index = int(len(X) * 0.7)  # 70% entrenamiento, 30% prueba
X_train, X_test = X.iloc[:split_index], X.iloc[split_index:]
y_train, y_test = y.iloc[:split_index], y.iloc[split_index:]

In [ ]:
df_completo.query('NDVI_mean>0.4')

,date,NDVI_mean,SAVI_mean,NDWI_mean,cloud_cover,system:index_x,system:index_y,precipitacion_diaria,system:time_start,.geo,temperatura_promedio,mes,trimestre,dia_del_año,NDVI_tendencia,temp_tendencia,NDVI_media_movil,floracion
10,2015-07-29,0.403166,0.604739,0.190025,24.61,LC08_026045_20150729,20150729,2.088112,1.438128e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",25.525125,7,3,210,0.086622,-0.097863,0.328972,0
11,2015-08-14,0.413895,0.620831,0.196603,5.87,LC08_026045_20150814,20150814,0.254287,1.439510e+12,"{""type"":""MultiPoint"",""coordinates"":[]}",26.466252,8,3,226,0.010729,0.941127,0.377868,0


In [ ]:

# Usar class_weight='balanced' si hay desbalance
modelo_rf = RandomForestClassifier(
    n_estimators=200,
    random_state=42,
    class_weight='balanced',  # Importante para series temporales
    max_depth=2               # Evitar overfitting
)

modelo_rf.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=2, n_estimators=200,
                       random_state=42)

In [ ]:
# 5. Evaluar el modelo
y_pred = modelo_rf.predict(X_test)
print("🎯 Accuracy:", accuracy_score(y_test, y_pred))
print("\n📊 Reporte de clasificación:")
print(classification_report(y_test, y_pred))

🎯 Accuracy: 0.896551724137931

📊 Reporte de clasificación:
              precision    recall  f1-score   support

           0       0.86      1.00      0.92        18
           1       1.00      0.73      0.84        11

    accuracy                           0.90        29
   macro avg       0.93      0.86      0.88        29
weighted avg       0.91      0.90      0.89        29



In [ ]:
prediccion=pd.DataFrame({"Actual":y_test, "Random Forest":y_pred})
prediccion.head()

,Actual,Random Forest
67,1,1
68,1,0
69,1,1
70,0,0
71,1,1


In [ ]:
print("✅ Forma de los datasets:")
print(f"X_train: {X_train.shape}, X_test: {X_test.shape}")
print(f"y_train: {y_train.shape}, y_test: {y_test.shape}")

print("\n✅ Rango de fechas:")
print(f"Entrenamiento: {df_completo['date'].iloc[X_train.index].min()} a {df_completo['date'].iloc[X_train.index].max()}")
print(f"Prueba: {df_completo['date'].iloc[X_test.index].min()} a {df_completo['date'].iloc[X_test.index].max()}")

✅ Forma de los datasets:
X_train: (67, 12), X_test: (29, 12)
y_train: (67,), y_test: (29,)

✅ Rango de fechas:
Entrenamiento: 2015-01-02 00:00:00 a 2018-07-05 00:00:00
Prueba: 2018-07-21 00:00:00 a 2019-12-15 00:00:00
